# Baby Names

*  Giulia MANNAIOLI
*  Lais Isabelle ALVES DOS SANTOS
*  Pedro ANDRADE FERREIRA SOBRINHO
*  Rafael SENNA BENATTI


### Import libraries

In [14]:
import altair as alt
import pandas as pd

alt.data_transformers.enable('json') # work-around to let Altair handle larger data sets

DataTransformerRegistry.enable('json')

### Data Loading

In [15]:
# Detect if using colab
if 'google.colab' in str(get_ipython()):
    colab = True
else:
    colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    path = '/content/drive/MyDrive/IGRMiniProject/baby_names_visualization/'
else:
    path = './'

In [40]:
# Reference to data
baby_names = pd.read_csv(path + 'dpt2020.csv', delimiter=';')

# Pre-process data set
baby_names = baby_names[baby_names.annais != 'XXXX']
baby_names = baby_names[baby_names.dpt != 'XX']

# show data
baby_names

,sexe,preusuel,annais,dpt,nombre
0,1,_PRENOMS_RARES,1900,02,7
1,1,_PRENOMS_RARES,1900,04,9
2,1,_PRENOMS_RARES,1900,05,8
3,1,_PRENOMS_RARES,1900,06,23
4,1,_PRENOMS_RARES,1900,07,9
...,...,...,...,...,...
3727545,2,ZYA,2013,44,4
3727546,2,ZYA,2013,59,3
3727547,2,ZYA,2017,974,3
3727548,2,ZYA,2018,59,3


### Visualization 1

How do baby names evolve over time? Are there names that have consistently remained popular or unpopular? Are there some that have were suddenly or briefly popular or unpopular? Are there trends in time?

In [50]:
# New data frame for visualization 1
baby_names_vis1 = baby_names
baby_names_vis1

,sexe,preusuel,annais,dpt,nombre
0,1,_PRENOMS_RARES,1900,02,7
1,1,_PRENOMS_RARES,1900,04,9
2,1,_PRENOMS_RARES,1900,05,8
3,1,_PRENOMS_RARES,1900,06,23
4,1,_PRENOMS_RARES,1900,07,9
...,...,...,...,...,...
3727545,2,ZYA,2013,44,4
3727546,2,ZYA,2013,59,3
3727547,2,ZYA,2017,974,3
3727548,2,ZYA,2018,59,3


In [54]:
def print_full_rows(x):
    """
    Print full rows of a dataset
    """
    pd.set_option('display.max_rows', None)
    print(x)
    pd.reset_option('display.max_rows')

In [58]:
# Sum the number of babies within the name and the year
n_names = baby_names_vis1.groupby(['annais', 'preusuel'])['nombre'].sum().reset_index()

# Create new dataframe with this data
new_baby_names_vis1 = pd.DataFrame({
    'names': n_names['preusuel'],
    'year': n_names['annais'],
    'number': n_names['nombre']
})

In [61]:
# Create visualization 1 with all the name
vis1 = alt.Chart(new_baby_names_vis1).mark_line().encode(
    x='year:T',         # encode the year as 'x' and time
    y='number:Q',       # encode the number of babies as 'y' and quantity
    color='names:N'     # encode the name as the color of the line
).properties(           
    # Change the size of the chart
    width=1000,
    height=500
)

vis1

alt.Chart(...)

Display random 3 names

In [67]:
# Choose three random names from the DataFrame
random_names = new_baby_names_vis1['names'].sample(n=3)
filtered_vis1 = new_baby_names_vis1[new_baby_names_vis1['names'].isin(random_names)]

In [66]:
# Call visualization
vis1 = alt.Chart(filtered_vis1).mark_line().encode(
    x='year:T',         # encode the year as 'x' and time
    y='number:Q',       # encode the number of babies as 'y' and quantity
    color='names:N'     # encode the name as the color of the line
).properties(           
    # Change the size of the chart
    width=1000,
    height=500
)

vis1

alt.Chart(...)

### Visualization 2

Is there a regional effect in the data? Are some names more popular in some regions? Are popular names generally popular across the whole country?

In [ ]:
import pandas as pd
import plotly.express as px

df_cleaned = baby_names.dropna(subset=['preusuel'])
selected_rows = df_cleaned[df_cleaned['preusuel'].str.contains('BRUNO')]
department_counts = selected_rows.groupby('dpt')['nombre'].sum().reset_index()

# Create a treemap using plotly
fig = px.treemap(department_counts, path=['dpt'], values='nombre')
fig.update_layout(title='Occurrences of Bruno by Department')
# Display the treemap
fig.show()

### Visualization 3

Are there gender effects in the data? Does popularity of names given to both sexes evolve consistently?

In [ ]:
import plotly.graph_objects as go

df_cleaned = baby_names.dropna(subset=['preusuel'])
name_year_counts = df_cleaned.groupby(['preusuel', 'annais'])['nombre'].sum().reset_index()

# Exclude names with preusuel = "_PRENOMS_RARES"
name_year_counts = name_year_counts[name_year_counts['preusuel'] != "_PRENOMS_RARES"]

# Filter for names for both sexes
name_counts = df_cleaned.groupby('preusuel')['sexe'].unique().reset_index()
filtered_names = name_counts[name_counts['sexe'].apply(lambda x: set(x) == {1, 2})]['preusuel']

# Sort by 'annais' (because if not the heatmap goes from 1962 to 2020 and then from 1904 to 1984)
name_year_counts_filtered = name_year_counts_filtered.sort_values('annais')


# Calculate the minimum and maximum popularity to set che colorscale
min_popularity = name_year_counts['nombre'].min()
max_popularity = name_year_counts['nombre'].max()

print("Minimum Popularity:", min_popularity)
print("Maximum Popularity:", max_popularity)

Minimum Popularity: 3
Maximum Popularity: 53596


In [ ]:

# Filter data for the year 2021
data_2021 = name_year_counts_filtered.loc[name_year_counts_filtered['annais'] == 2022]
data_2021


,preusuel,annais,nombre


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Define the logarithmic range of values
zmin = 3
zmax = 50000
z_range = np.logspace(np.log10(zmin), np.log10(zmax), len(colorscales)-1)

colorscales = [
    [0.0, "rgb(68, 1, 84)"],
    [0.03, "rgb(72, 35, 116)"],
    [0.07, "rgb(64, 86, 135)"],
    [0.1, "rgb(52, 132, 144)"],
    [0.13, "rgb(94, 181, 122)"],
    [0.17, "rgb(140, 208, 95)"],
    [0.2, "rgb(179, 226, 70)"],
    [0.23, "rgb(217, 240, 52)"],
    [0.27, "rgb(253, 231, 37)"],
    [0.3, "rgb(238, 160, 30)"],
    [1.0, "rgb(215, 25, 28)"]
]

# Create the heatmap chart using plotly
fig = go.Figure(data=go.Heatmap(
    x=name_year_counts_filtered['annais'],
    y=name_year_counts_filtered['preusuel'],
    z=name_year_counts_filtered['nombre'],
    colorscale=colorscales,
    zmin=zmin,
    zmax=zmax,
))

# Customize the layout of the heatmap
fig.update_layout(
    title='Popularity of Baby Neutral-Names Over Time',
    xaxis_title='Year',
    yaxis_title='Names',
)

# Update the hover label formatting
fig.update_traces(hovertemplate='Year: %{x}<br>'+
                                  'Names: %{y}<br>'+
                                  'Popularity: %{z}')

# Add colorbar
fig.update_layout(coloraxis_colorbar=dict(
    title='Popularity',
    titleside='right',
    tickmode='array',
    tickvals=z_range,
    ticktext=[f'{int(val)}' if val >= 1000 else f'{int(val):.0f}' for val in z_range]
))

# Display the heatmap
fig.show()



                                              0.0/15.5 MB ? eta -:--:--
                                              0.3/15.5 MB 5.9 MB/s eta 0:00:03
     -                                        0.7/15.5 MB 6.6 MB/s eta 0:00:03
     --                                       1.0/15.5 MB 6.1 MB/s eta 0:00:03
     ---                                      1.2/15.5 MB 6.0 MB/s eta 0:00:03
     ----                                     1.6/15.5 MB 6.4 MB/s eta 0:00:03
     ----                                     1.9/15.5 MB 6.5 MB/s eta 0:00:03
     -----                                    2.2/15.5 MB 6.5 MB/s eta 0:00:03
     ------                                   2.6/15.5 MB 6.6 MB/s eta 0:00:02
     -------                                  3.0/15.5 MB 6.7 MB/s eta 0:00:02
     --------                                 3.3/15.5 MB 6.8 MB/s eta 0:00:02
     ---------                                3.6/15.5 MB 6.8 MB/s eta 0:00:02
     ----------                               4.0/15.5 MB 6

- With the hover, when there is no name in a given year, the heatmap does not display any color or z data. However, it is important to note that in these cases, the popularity of the name is actually 0. Unfortunately, I was unable to find a solution to display the value 0 in the hover label when there is no corresponding data for a specific name and year.

- In an attempt to address the issue of color gradient, I made modifications to the colorscale used in the heatmap. Previously, the colorscale was dominated by shades of purple, making it difficult to distinguish variations in popularity for names with counts less then 10,000. To enhance the visual representation, I expanded the colorscale to include a wider range of colors for counts up to 15,000, and reduced the number of colors used for counts above that threshold.

- To navigate the heatmap effectively, it is recommended to zoom in and out of the chart. This allows for a better view and legibility of all the names displayed. By adjusting the zoom level, users can explore the heatmap in detail and observe the changing popularity of names over time.

- It is worth noting that the year 2021 may appear unusual in the heatmap. This is because, upon hovering, it does not display the names or corresponding popularity data as expected. Additionally, there are numerous names that appear for the first time in this  year. Further exploration and analysis may be required to fully understand the patterns observed in the year 2021.


### Comments

We choose to implement the heatmap because this visualization effectively showcases the evolution of baby neutral-names over time.
Compared to other visualizations, such as line charts or bar graphs, the heatmap has distinct advantages. It excels in showcasing the complete picture without overlapping lines, ensuring that each name's popularity is clearly visible and discernible. By utilizing varying colors, the heatmap not only presents the popularity of each name but also enables easy identification and understanding of the trends. This makes it an ideal choice for analyzing the evolution of baby neutral-names, as it provides a comprehensive and visually appealing representation of the data.
Anyway, it's important to consider factors such as the size limitations of the visualization with a large number of names and challenges in distinguishing between subtle variations in quantity.

In analyzing the data, it is evident that there are gender effects and trends in the popularity of names given to both sexes. Over time, there has been a significant increase of giving neutral names to children. This trend is particularly pronounced in the last period, where we observe a consistent and linear growth in the popularity of neutral names, especially after the **year 1950** (more or less).

This increase in the use of neutral names can be attributed to several factors. Firstly, the progress in LGBT+ rights and the increasing acceptance and recognition of diverse gender identities have influenced the naming practices. Additionally, the empowerment of women and the blurring of traditional gender roles have also contributed to the rise of neutral names. As societal norms evolve and the distinctions between genders become less rigid, there is a growing tendency to choose names that are not strictly associated with a specific gender.
So, the data supports the notion that the popularity of neutral names has been evolving consistently, with a notable surge in recent years. This trend reflects the broader societal changes towards inclusivity and the recognition of diverse gender expressions.